In [ ]:
!pip install datasets torchaudio transformers

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.2/491.2 kB 8.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 64.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 71.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 37.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 12.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 78.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 8.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
from google.colab import drive
drive.mount('/content/drive')



Mounted at /content/drive


In [ ]:
!rm -r dataset/

rm: cannot remove 'dataset/': No such file or directory


In [ ]:
# Unzip the file
import zipfile


# Specify the path to your file in Drive
zip_file_path = '/content/drive/My Drive/Speech_to_text/dataset_10/dataset_10.zip'


with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
    zip_ref.extractall('/content/dataset/')

In [ ]:
from datasets import load_from_disk


# Load the dataset from the saved directory
dataset = load_from_disk('/content/dataset')

dataset

Dataset({
    features: ['client_id', 'path', 'audio', 'sentence', 'up_votes', 'down_votes', 'age', 'gender', 'accent', 'locale', 'segment'],
    num_rows: 94873
})

In [ ]:
train_val_test_split = dataset.train_test_split(test_size=0.2, shuffle = True, seed = 42) # 80% train, 20% val+test
train_split = train_val_test_split['train']
val_test_split = train_val_test_split['test'].train_test_split(test_size=0.5, shuffle = True, seed = 42)
val_split = val_test_split['train']
test_split = val_test_split['test']

In [ ]:
print(len(train_split))
print(len(val_split))
print(len(test_split))

75898
9487
9488


In [ ]:
import random
import pandas as pd
from IPython.display import display, HTML

def show_random_elements_streaming(dataset, num_examples=10):
    sampled_data = []

    for i, example in enumerate(dataset):  # Iterate over dataset
        if len(sampled_data) < num_examples:
            sampled_data.append(example)
        else:
            break  # Stop when we have enough examples

    df = pd.DataFrame(sampled_data)
    display(HTML(df.to_html()))

# Call the function on your streaming dataset
show_random_elements_streaming(train_split)


,client_id,path,audio,sentence,up_votes,down_votes,age,gender,accent,locale,segment
0,af294a21d920e11013661f4735c1ebd5df298b0c73074b35ae4c3ea490ee8e630766fc629741346f123353256859ef7d055ee8dcce07e48a091cc8bedec671e3,/root/.cache/huggingface/datasets/downloads/extracted/47b1deb7d3dfe1d63f58de453d18fb94cb752b21d9e7e4c397cd3b3939f970ce/en_train_18/common_voice_en_27223621.mp3,"{'path': 'common_voice_en_27223621.mp3', 'array': [2.5011104298755527e-12, 4.888534022029489e-12, 1.0231815394945443e-12, -5.5706550483591855e-12, -5.9685589803848416e-12, -2.0463630789890885e-12, 2.7284841053187847e-12, 6.366462912410498e-12, 6.366462912410498e-12, 1.659827830735594e-11, 3.3026026358129457e-11, 2.7057467377744615e-11, 0.0, -1.4551915228366852e-11, -5.229594535194337e-12, 5.9117155615240335e-12, 1.4551915228366852e-11, 1.3642420526593924e-11, -3.637978807091713e-12, -2.8421709430404007e-11, -4.547473508864641e-11, -6.070877134334296e-11, -6.730260793119669e-11, -4.547473508864641e-11, -4.547473508864641e-13, 5.070432962384075e-11, 9.481482265982777e-11, 1.1004885891452432e-10, 8.822098607197404e-11, 5.002220859751105e-11, 1.0686562745831907e-11, -3.0240698833949864e-11, -5.547917680814862e-11, -3.149125404888764e-11, 3.524291969370097e-11, 6.752998160663992e-11, -6.849631972727366e-12, -1.7712409317027777e-10, -2.944489096989855e-10, -2.5579538487363607e-10, -1.2676082405960187e-10, -5.616129783447832e-11, -4.7975845518521965e-11, 1.864464138634503e-11, 1.4733814168721437e-10, 1.5575096767861396e-10, -1.0913936421275139e-11, -5.547917680814862e-11, 3.0252067517722026e-10, 8.133156370604411e-10, 9.088125807465985e-10, 4.447997525858227e-10, -4.3655745685100555e-11, -2.4556356947869062e-11, 3.001332515850663e-10, 3.0377123039215803e-10, -9.913492249324918e-11, -2.810338628478348e-10, 7.935341272968799e-11, 3.8107828004285693e-10, -3.183231456205249e-12, -7.307789928745478e-10, -9.1313268058002e-10, -3.5197444958612323e-10, 1.5199930203380063e-10, -1.5552359400317073e-10, -9.617906471248716e-10, -1.2760210665874183e-09, -7.200924301287159e-10, 3.257127900724299e-11, 1.8712853488977998e-10, -2.071374183287844e-10, -4.233697836752981e-10, -2.1827872842550278e-11, 4.845333023695275e-10, 1.0959411156363785e-10, -1.291482476517558e-09, -2.4606379156466573e-09, -2.106389729306102e-09, -6.614300218643621e-10, 2.5920599000528455e-11, -9.44510247791186e-10, -2.19279172597453e-09, -1.8767423171084374e-09, -4.3309000830049627e-10, -1.2607870303327218e-10, -1.5793375496286899e-09, -2.52907739195507e-09, -1.1295924196019769e-09, 9.858922567218542e-10, 9.608811524230987e-10, -8.108145266305655e-10, -8.999236911222397e-10, 2.1129835658939555e-09, 5.1344954954402056e-09, 4.572939360514283e-09, 1.2560121831484139e-09, -8.824372343951836e-10, -2.2464519133791327e-10, 9.927134669851512e-10, ...], 'sampling_rate': 48000}",She was named after Admiral of the Fleet Sir John Kelly.,2,0,teens,female,"India and South Asia (India, Pakistan, Sri Lanka)",en,
1,be72cb64d8918a11501ef152a39ae3c39e237361f4dee91611e7195dde75d76b07351a9d851f1788d375888fad90c004e2581ca4937fb50c3c6b8e0e22e4b259,/root/.cache/huggingface/datasets/downloads/extracted/c7b9900813cdacaf23599a659b0db28cca013c4f7bbdc977dc723c4f62295775/en_train_7/common_voice_en_24977460.mp3,"{'path': 'common_voice_en_24977460.mp3', 'array': [3.226004388956808e-23, 1.6543612251060553e-24, -1.406207041340147e-23, -2.1299900773240463e-23, 4.1359030627651384e-23, 4.218621124020441e-23, 1.3234889800848443e-23, -2.3794367307970687e-23, -5.37667398159468e-24, 1.9645539548134407e-24, 1.4372263143108856e-23, 1.737079286361358e-23, 1.6543612251060553e-24, 1.1580528575742387e-23, -4.1359030627651384e-24, 7.548023089546378e-24, -1.6543612251060553e-23, -2.3678045034330417e-23, -2.9015944924711674e-23, -1.1683926152311516e-23, 2.5125611106298216e-23, -1.7422491651898145e-23, -1.4475660719677984e-23, -2.5952791718851243e-23, 3.5155176033503676e-24, -3.7223127564886245e-24, -3.825710333057753e-24, 1.406207041340147e-23, 1.0339757656912

In [ ]:
datapoint = train_split[0]

print(len(datapoint['audio']['array']))
print(len(datapoint['sentence']))

390528
56


In [ ]:
datapoint = train_split[1]

print(len(datapoint['audio']['array']))
print(len(datapoint['sentence']))

302400
65


#NOTE: the length of audio signal is different for each sample as well as the length of the sentence is different

#NOTE: The length of teh audio signal is much larger than the length of the sentence

#NOTE: The sentences have punctuations!

#1] Data Preprocessing




STEP 1: Resampling at 16kHz for Wave2vec

NOTE: The below code takes 10-15 mins to run

In [ ]:
from datasets import Audio

train_split = train_split.cast_column('audio', Audio(sampling_rate=16_000))
val_split = val_split.cast_column('audio', Audio(sampling_rate=16_000))
test_split = test_split.cast_column('audio', Audio(sampling_rate=16_000))


def preprocess(batch):
  batch['input_samples'] = [audio['array'] for audio in batch['audio']]
  batch['input_length'] = [len(audio["array"]) for audio in batch["audio"]]
  return batch

train_split = train_split.map(preprocess, remove_columns=['audio'], batched=True, batch_size=1000)
val_split = val_split.map(preprocess, remove_columns=['audio'], batched=True, batch_size=1000)
test_split = test_split.map(preprocess, remove_columns=['audio'], batched=True, batch_size=1000)

Map:   0%|          | 0/75898 [00:00<?, ? examples/s]

Map:   0%|          | 0/9487 [00:00<?, ? examples/s]

Map:   0%|          | 0/9488 [00:00<?, ? examples/s]

In [ ]:
import numpy as np

print("Max audio length = ", max(train_split['input_length']))
print("Min audio length = ", min(train_split['input_length']))
print("Avg audio length = ", np.mean(train_split['input_length']))


print("max length of audio in seconds = ", max(train_split['input_length'])/16000)
print("min length of audio in seconds = ", min(train_split['input_length'])/16000)
print("avg length of audio in seconds = ", np.mean(train_split['input_length'])/16000)


Max audio length =  798912
Min audio length =  19584
Avg audio length =  91285.18848981528
max length of audio in seconds =  49.932
min length of audio in seconds =  1.224
avg length of audio in seconds =  5.705324280613455


In [ ]:
print("Total samples = ", len(train_split),"\n")

#### lets check distribution

# Calculate percentiles
percentiles = [25, 50, 75, 90, 95, 99]  # 25th, 50th, 75th percentiles
percentile_values = np.percentile(train_split['input_length'], percentiles)

print(f"25th percentile: {percentile_values[0]}")
print(f"50th percentile (Median): {percentile_values[1]}")
print(f"75th percentile: {percentile_values[2]}")
print(f"90th percentile: {percentile_values[3]}")
print(f"95th percentile: {percentile_values[4]}")
print(f"99th percentile: {percentile_values[5]}")

# foll takes too long to run
#print(f"\nPercentage of samples below 25th percentile = {len(train_split[train_split['input_length'] < percentile_values[0]])/len(train_split)*100}")
#print(f"Percentage of samples below 50th percentile = {len(train_split[train_split['input_length'] < percentile_values[1]])/len(train_split)*100}")
#print(f"Percentage of samples below 75th percentile = {len(train_split[train_split['input_length'] < percentile_values[2]])/len(train_split)*100}")

Total samples =  75898 

25th percentile: 72576.0
50th percentile (Median): 89280.0
75th percentile: 107712.0
90th percentile: 125568.0
95th percentile: 137088.0
99th percentile: 155531.51999999955


In [ ]:
train_split[0]

{'client_id': 'af294a21d920e11013661f4735c1ebd5df298b0c73074b35ae4c3ea490ee8e630766fc629741346f123353256859ef7d055ee8dcce07e48a091cc8bedec671e3',
 'path': '/root/.cache/huggingface/datasets/downloads/extracted/47b1deb7d3dfe1d63f58de453d18fb94cb752b21d9e7e4c397cd3b3939f970ce/en_train_18/common_voice_en_27223621.mp3',
 'sentence': 'She was named after Admiral of the Fleet Sir John Kelly.',
 'up_votes': 2,
 'down_votes': 0,
 'age': 'teens',
 'gender': 'female',
 'accent': 'India and South Asia (India, Pakistan, Sri Lanka)',
 'locale': 'en',
 'segment': '',
 'input_samples': [-2.9831426218152046e-10,
  -3.92901711165905e-10,
  -5.675246939063072e-10,
  -2.9103830456733704e-10,
  2.9103830456733704e-11,
  2.3283064365386963e-10,
  2.255546860396862e-10,
  1.7462298274040222e-10,
  4.3655745685100555e-11,
  -2.1827872842550278e-10,
  -3.637978807091713e-10,
  0.0,
  5.820766091346741e-11,
  4.3655745685100555e-11,
  3.055902197957039e-10,
  2.3283064365386963e-10,
  5.966285243630409e-10,
  

In [ ]:
# Specify the path where you want to save the dataset
save_path = '/content/drive/My Drive/Speech_to_text/dataset_10/second_colab_processing/after_1'

# Save the dataset to Google Drive
train_split.save_to_disk(f"{save_path}/train")
val_split.save_to_disk(f"{save_path}/val")
test_split.save_to_disk(f"{save_path}/test")


Saving the dataset (0/111 shards):   0%|          | 0/75898 [00:00<?, ? examples/s]

Saving the dataset (0/14 shards):   0%|          | 0/9487 [00:00<?, ? examples/s]

Saving the dataset (0/14 shards):   0%|          | 0/9488 [00:00<?, ? examples/s]

# START 1

In [ ]:
### lets load from drive now

from datasets import load_from_disk

save_path = '/content/drive/My Drive/Speech_to_text/dataset_10/second_colab_processing/after_1'

train_split = load_from_disk(f"{save_path}/train")
val_split = load_from_disk(f"{save_path}/val")
test_split = load_from_disk(f"{save_path}/test")

print(train_split)
print(val_split)
print(test_split)


Loading dataset from disk:   0%|          | 0/111 [00:00<?, ?it/s]

Dataset({
    features: ['client_id', 'path', 'sentence', 'up_votes', 'down_votes', 'age', 'gender', 'accent', 'locale', 'segment', 'input_samples', 'input_length'],
    num_rows: 75898
})
Dataset({
    features: ['client_id', 'path', 'sentence', 'up_votes', 'down_votes', 'age', 'gender', 'accent', 'locale', 'segment', 'input_samples', 'input_length'],
    num_rows: 9487
})
Dataset({
    features: ['client_id', 'path', 'sentence', 'up_votes', 'down_votes', 'age', 'gender', 'accent', 'locale', 'segment', 'input_samples', 'input_length'],
    num_rows: 9488
})


NOTE: It seems that are some files where Audio is completely 0.

Hence

1] removing entries where audio values are < 1e-6

2] removeing rows where sentences are null

3] removing punctuations and making upper case. this is since

In [ ]:
import numpy as np
import re

# Function to check for silent audio
def is_silent(audio_array, threshold=1e-6):
    return np.all(np.abs(audio_array) < threshold)

# Function to check for valid text
def has_valid_text(batch):
    return batch["sentence"] and not batch["sentence"].isspace()

# Function to normalize text
def normalize_text(text):
    text = text.upper()  # Wav2Vec2 works better with uppercase
    text = re.sub(r"[^A-Z' ]", "", text)  # Keep only letters, spaces, and apostrophes --> lets create a vocab with the punctuations and see
    return text.strip()





In [ ]:
# Apply filtering and text normalization for train
train_split = train_split.filter(lambda x: not is_silent(x["input_samples"]))  # Remove silent audio
train_split = train_split.filter(has_valid_text)  # Remove empty sentences
train_split = train_split.map(lambda x: {"sentence": normalize_text(x["sentence"])})  # Normalize text


Filter:   0%|          | 0/75898 [00:00<?, ? examples/s]

Filter:   0%|          | 0/75898 [00:00<?, ? examples/s]

Map:   0%|          | 0/75898 [00:00<?, ? examples/s]

In [ ]:
# Specify the path where you want to save the dataset
save_path = '/content/drive/My Drive/Speech_to_text/dataset_10/second_colab_processing/after_filtering_and_normalizing'

# Save the dataset to Google Drive
train_split.save_to_disk(f"{save_path}/train")



Saving the dataset (0/111 shards):   0%|          | 0/75898 [00:00<?, ? examples/s]

In [ ]:
# Apply filtering and text normalization for val
val_split = val_split.filter(lambda x: not is_silent(x["input_samples"]))
val_split = val_split.filter(has_valid_text)
val_split = val_split.map(lambda x: {"sentence": normalize_text(x["sentence"])})



Filter:   0%|          | 0/9487 [00:00<?, ? examples/s]

Filter:   0%|          | 0/9487 [00:00<?, ? examples/s]

Map:   0%|          | 0/9487 [00:00<?, ? examples/s]

In [ ]:
# Specify the path where you want to save the dataset
save_path = '/content/drive/My Drive/Speech_to_text/dataset_10/second_colab_processing/after_filtering_and_normalizing'

# Save the dataset to Google Drive
val_split.save_to_disk(f"{save_path}/val")

Saving the dataset (0/14 shards):   0%|          | 0/9487 [00:00<?, ? examples/s]

In [ ]:
# Apply filtering and text normalization for test
test_split = test_split.filter(lambda x: not is_silent(x["input_samples"]))
test_split = test_split.filter(has_valid_text)
test_split = test_split.map(lambda x: {"sentence": normalize_text(x["sentence"])})

Filter:   0%|          | 0/9488 [00:00<?, ? examples/s]

Filter:   0%|          | 0/9488 [00:00<?, ? examples/s]

Map:   0%|          | 0/9488 [00:00<?, ? examples/s]

In [ ]:
# Specify the path where you want to save the dataset
save_path = '/content/drive/My Drive/Speech_to_text/dataset_10/second_colab_processing/after_filtering_and_normalizing'

# Save the dataset to Google Drive
test_split.save_to_disk(f"{save_path}/test")

Saving the dataset (0/14 shards):   0%|          | 0/9488 [00:00<?, ? examples/s]

#Start 2

In [ ]:
### lets load from drive now

from datasets import load_from_disk

save_path = '/content/drive/My Drive/Speech_to_text/dataset_10/second_colab_processing/after_filtering_and_normalizing'

train_split = load_from_disk(f"{save_path}/train")
val_split = load_from_disk(f"{save_path}/val")
test_split = load_from_disk(f"{save_path}/test")

print(train_split)
print(val_split)
print(test_split)


Loading dataset from disk:   0%|          | 0/111 [00:00<?, ?it/s]

Dataset({
    features: ['client_id', 'path', 'sentence', 'up_votes', 'down_votes', 'age', 'gender', 'accent', 'locale', 'segment', 'input_samples', 'input_length'],
    num_rows: 75898
})
Dataset({
    features: ['client_id', 'path', 'sentence', 'up_votes', 'down_votes', 'age', 'gender', 'accent', 'locale', 'segment', 'input_samples', 'input_length'],
    num_rows: 9487
})
Dataset({
    features: ['client_id', 'path', 'sentence', 'up_votes', 'down_votes', 'age', 'gender', 'accent', 'locale', 'segment', 'input_samples', 'input_length'],
    num_rows: 9488
})


In [ ]:
import random
import pandas as pd
from IPython.display import display, HTML

def show_random_elements_streaming(dataset, num_examples=10):
    sampled_data = []

    for i, example in enumerate(dataset):  # Iterate over dataset
        if len(sampled_data) < num_examples:
            sampled_data.append(example)
        else:
            break  # Stop when we have enough examples

    df = pd.DataFrame(sampled_data)
    display(HTML(df.to_html()))

# Call the function on your streaming dataset
show_random_elements_streaming(train_split)

,client_id,path,sentence,up_votes,down_votes,age,gender,accent,locale,segment,input_samples,input_length
0,af294a21d920e11013661f4735c1ebd5df298b0c73074b35ae4c3ea490ee8e630766fc629741346f123353256859ef7d055ee8dcce07e48a091cc8bedec671e3,/root/.cache/huggingface/datasets/downloads/extracted/47b1deb7d3dfe1d63f58de453d18fb94cb752b21d9e7e4c397cd3b3939f970ce/en_train_18/common_voice_en_27223621.mp3,SHE WAS NAMED AFTER ADMIRAL OF THE FLEET SIR JOHN KELLY,2,0,teens,female,"India and South Asia (India, Pakistan, Sri Lanka)",en,,"[-2.9831426218152046e-10, -3.92901711165905e-10, -5.675246939063072e-10, -2.9103830456733704e-10, 2.9103830456733704e-11, 2.3283064365386963e-10, 2.255546860396862e-10, 1.7462298274040222e-10, 4.3655745685100555e-11, -2.1827872842550278e-10, -3.637978807091713e-10, 0.0, 5.820766091346741e-11, 4.3655745685100555e-11, 3.055902197957039e-10, 2.3283064365386963e-10, 5.966285243630409e-10, 6.111804395914078e-10, -2.4010660126805305e-10, 8.003553375601768e-11, -6.548361852765083e-10, -2.1827872842550278e-11, -5.675246939063072e-10, -1.8189894035458565e-10, 4.802132025361061e-10, -5.820766091346741e-10, -1.7462298274040222e-09, -8.149072527885437e-10, -1.6880221664905548e-09, -1.1496013030409813e-09, -1.6007106751203537e-10, 2.5029294192790985e-09, 2.8812792152166367e-09, -6.111804395914078e-10, 1.2223608791828156e-09, -4.48198989033699e-09, -8.731149137020111e-10, -3.7834979593753815e-10, -1.760781742632389e-09, 2.6775524020195007e-09, 6.402842700481415e-10, -1.6916601452976465e-09, -3.14321368932724e-09, 7.712515071034431e-10, -5.820766091346741e-11, -4.9622030928730965e-09, 2.5174813345074654e-09, -3.3760443329811096e-09, 1.979060471057892e-09, 1.1641532182693481e-10, -2.735760062932968e-09, -1.4646502677351236e-08, -1.752050593495369e-08, -5.326000973582268e-09, -2.342858351767063e-08, -3.6088749766349792e-09, -3.899913281202316e-09, -6.05359673500061e-09, -1.7636921256780624e-08, -7.30506144464016e-09, -2.306478563696146e-08, -1.05355866253376e-08, -4.554749466478825e-09, 3.4924596548080444e-09, 1.2631062418222427e-08, -1.0506482794880867e-08, 2.2031599655747414e-08, -4.190951585769653e-08, 6.344635039567947e-09, 1.0826624929904938e-08, -3.739842213690281e-08, 1.234002411365509e-08, 4.656612873077393e-10, 1.457010512240231e-08, -3.172317519783974e-08, 8.17817635834217e-09, -1.775333657860756e-09, -6.171467248350382e-08, 4.030880518257618e-08, -4.691537469625473e-08, 7.225025910884142e-09, -4.2913598008453846e-08, -3.731111064553261e-08, -1.3405224308371544e-07, -1.2926830095238984e-07, -2.5247572921216488e-08, -1.3176759239286184e-07, -1.5956175047904253e-08, -8.321512723341584e-08, -1.210719347000122e-08, -1.6824924387037754e-07, -3.518653102219105e-08, -2.1877349354326725e-07, -2.1615414880216122e-07, -3.746390575543046e-08, -1.9858998712152243e-07, -9.83782229013741e-08, -1.1686643119901419e-07, 1.6429112292826176e-07, -6.57601049169898e-08, ...]",130176
1,be72cb64d8918a11501ef152a39ae3c39e237361f4dee91611e7195dde75d76b07351a9d851f1788d375888fad90c004e2581ca4937fb50c3c6b8e0e22e4b259,/root/.cache/huggingface/datasets/downloads/extracted/c7b9900813cdacaf23599a659b0db28cca013c4f7bbdc977dc723c4f62295775/en_train_7/common_voice_en_24977460.mp3,THERE ARE CURRENTLY THREE FULLTIME DEEMSTERS IN THE ISLE OF MAN,2,0,,,,en,,"[-3.3306690738754696e-16, 3.3306690738754696e-15, -1.5543122344752192e-15, 1.5543122344752192e-15, -6.661338147750939e-16, 1.1102230246251565e-15, -1.5265566588595902e-15, 0.0, 2.220446049250313e-16, 1.5543122344752192e-15, 0.0, 9.992007221626409e-16, 7.771561172376096e-16, 2.1094237467877974e-15, 1.5543122344752192e-15, -6.661338147750939e-16, 5.551115123125783e-16, -1.3322676295501878e-15, -1.9984014443252818e-15, -6.661338147750939e-16, 1.2212453270876722e-15, 6.106226635438361e-16, -2.220446049250313e-16, 2.220446049250313e-16, -4.440892098500626e-16, -6.661338147750939e-16, 8.881784197001252e-16, -3.9968028886505635e-15, -1.3322676295501878e-15, -6.661338147750939e-16, -2.220446049250313e-16, -5.

In [ ]:
train_split[9]

{'client_id': '48f845f319f815c056993d8b890c080bd017fdde1ac54129bd1acfec3cb99c0c60cbec4d6a1b27db52d5b9ce84e4da9b600504e033b2398503a7243297f52a31',
 'path': '/root/.cache/huggingface/datasets/downloads/extracted/7dc1609c34ac7013a81d779000ef5208a40c546b73a7e071e7e2f57355ed1a8c/en_train_0/common_voice_en_20051835.mp3',
 'sentence': 'THE MAGAZINE HAS ITS HEADQUARTERS IN BEIJING',
 'up_votes': 2,
 'down_votes': 0,
 'age': '',
 'gender': '',
 'accent': '',
 'locale': 'en',
 'segment': '',
 'input_samples': [0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,

In [ ]:
all_text = " ".join(train_split["sentence"])  # Join all sentences into one string
vocab_list = list(set(all_text))  # Extract unique characters


In [ ]:
vocab_dict = {v: k for k, v in enumerate(vocab_list)}
vocab_dict

{'S': 0,
 'U': 1,
 ' ': 2,
 'X': 3,
 'P': 4,
 'K': 5,
 'V': 6,
 'B': 7,
 'Z': 8,
 'A': 9,
 'I': 10,
 'E': 11,
 'N': 12,
 'T': 13,
 'M': 14,
 'L': 15,
 'R': 16,
 'Y': 17,
 'G': 18,
 'W': 19,
 'F': 20,
 'C': 21,
 "'": 22,
 'D': 23,
 'O': 24,
 'J': 25,
 'Q': 26,
 'H': 27}

In [ ]:

vocab_dict["|"] = vocab_dict[" "]
del vocab_dict[" "]

vocab_dict["[UNK]"] = len(vocab_dict)
vocab_dict["[PAD]"] = len(vocab_dict)
vocab_dict

{'S': 0,
 'U': 1,
 'X': 3,
 'P': 4,
 'K': 5,
 'V': 6,
 'B': 7,
 'Z': 8,
 'A': 9,
 'I': 10,
 'E': 11,
 'N': 12,
 'T': 13,
 'M': 14,
 'L': 15,
 'R': 16,
 'Y': 17,
 'G': 18,
 'W': 19,
 'F': 20,
 'C': 21,
 "'": 22,
 'D': 23,
 'O': 24,
 'J': 25,
 'Q': 26,
 'H': 27,
 '|': 2,
 '[UNK]': 28,
 '[PAD]': 29}

In [ ]:
import json

with open('./vocab.json', 'w') as f:
  json.dump(vocab_dict, f)

In [ ]:
import shutil

# Define the source and destination paths
source_path = './vocab.json'
destination_path = '/content/drive/My Drive/Speech_to_text/dataset_10/second_colab_processing/vocab.json'

# Copy the file
shutil.copy(source_path, destination_path)

print(f"File successfully copied to: {destination_path}")


File successfully copied to: /content/drive/My Drive/Speech_to_text/dataset_10/second_colab_processing/vocab.json


----
#NOTE: 75 % data falls below length 107712. The sampling rate is 16000 Hz which mean one second has 16k samples.

# And 99 % data falls below 155531.51999999955

# Formula for sample length = number of seconds * Sampling_frequency

# so 10 seconds = 160000 samples

# Setting max_length to 160000


---

Ah, I see! You're right to question the overlap between those two steps. Let me clarify the difference:

1. **`dataset.cast_column("audio", Audio(sampling_rate=16_000))`**:
   - This step **resamples** the audio files in the dataset to a uniform sampling rate of 16 kHz.
   - It's a **data-level operation** that ensures all the audio is at the correct sampling rate before passing it to the model. It simply standardizes the audio to the required format, but doesn't apply any other transformations.
   - After this step, the audio data is ready for further processing but still in its raw form (waveform).

2. **`batch["input_values"] = processor(batch["input_values"], sampling_rate=16_000).input_values[0]`**:
   - This is a **feature extraction** step, and not just resampling.
   - The `processor` does more than just resample the audio — it converts the raw waveform into a representation the model can use (such as a **spectrogram** or **mel-spectrogram**). In short, it prepares the audio data for the actual model.
   - The `sampling_rate` here is provided for **correct processing** (ensuring that the processor correctly interprets the audio data after resampling).
   - Additionally, this step applies any **preprocessing** necessary for the specific model (such as normalization or tokenization of the input values).

### Key Differences:

- **`cast_column`**: Only resamples the audio to the correct sampling rate (no feature extraction, just ensures uniformity).
- **`processor`**: Resamples the audio and also does **feature extraction** (turns the raw waveform into features like spectrograms or MFCCs), preparing the data for the model.

### TL;DR:
While both steps deal with the **sampling rate**, the `cast_column` step is just for **uniform resampling** (standardizing the audio format), while the `processor` step **converts the raw audio into the actual features** that the model can work with. So, they serve different purposes — one for data consistency, the other for feature extraction.

#Step 2: Extract Features

### **Wav2Vec2-Large-960h (facebook/wav2vec2-large-960h)**  

This is a **pretrained Automatic Speech Recognition (ASR) model** developed by **Facebook (Meta)**.  

#### **Key Features:**  
✅ **Pretrained on 960 hours of Librispeech** → Trained on high-quality **English** speech data  
✅ **Self-Supervised Learning** → Learns speech representations without labeled data, then fine-tuned for ASR  
✅ **CTC (Connectionist Temporal Classification)** → Predicts characters or words without needing alignment  
✅ **Works on 16kHz audio** → Requires resampling for consistency  

#### **How it Works:**  
1. **Feature Extraction** → Converts raw waveform into meaningful speech representations  
2. **Transformer Encoder** → Processes audio features to understand speech patterns  
3. **CTC Decoding** → Outputs transcriptions without frame-level alignment  

#### **When to Use It?**  
- If you need **English speech-to-text**  
- Works best with **clean and well-recorded** speech  
- Can be **fine-tuned** on your dataset for better accuracy  

🚀 **Want something better for noisy/multilingual audio?**  
- Try `"facebook/wav2vec2-large-robust"` (for noisy environments)  
- Try `"openai/whisper"` (for multi-language + noise handling)  


# Why Fine-Tune if thsi model has alreday been trained for ASR?


Even though **facebook/wav2vec2-large-960h** is already trained on 960 hours of **Librispeech**, fine-tuning it on **Common Voice** (or any dataset) improves performance because:  

### **Why Fine-Tune?**  
1️⃣ **Domain Adaptation** → Librispeech is mainly **audiobooks**, but Common Voice has **diverse accents, casual speech, and background noise**. Fine-tuning makes the model **better suited** for your specific use case.  

2️⃣ **Better Generalization** → Pretrained models work well **generally**, but adding more real-world examples **reduces errors** for different pronunciations, speech styles, and recording conditions.  

3️⃣ **Specific Vocabulary** → If your dataset has **unique words, names, or technical terms** not covered in the original training (e.g., medical terms, slang, etc.), fine-tuning helps the model learn those.  

4️⃣ **Accent & Speaker Variability** → Librispeech is mainly **American English**, but Common Voice includes **various accents** (British, Australian, Indian, etc.). Fine-tuning helps the model understand more speakers accurately.  

If your dataset is **very similar** to Librispeech, you might not need much fine-tuning. But if you're dealing with **casual speech, different accents, or noisy environments**, fine-tuning can make a huge difference! 🚀

### Step 2 - Start

In [ ]:
from transformers import Wav2Vec2CTCTokenizer

tokenizer = Wav2Vec2CTCTokenizer("./vocab.json", unk_token="[UNK]", pad_token="[PAD]", word_delimiter_token="|")


In [ ]:
### push tokenizer to the repo

from huggingface_hub import notebook_login

notebook_login()





In [ ]:
repo = 'ASR-Common_speech_11_10'
tokenizer.push_to_hub(repo)

CommitInfo(commit_url='https://huggingface.co/Tanaya25/ASR-Common_speech_11_10/commit/2050c73534d4f7e108bd9a6ffe867cd932c6344d', commit_message='Upload tokenizer', commit_description='', oid='2050c73534d4f7e108bd9a6ffe867cd932c6344d', pr_url=None, repo_url=RepoUrl('https://huggingface.co/Tanaya25/ASR-Common_speech_11_10', endpoint='https://huggingface.co', repo_type='model', repo_id='Tanaya25/ASR-Common_speech_11_10'), pr_revision=None, pr_num=None)

We no define rge feature extractor

 Wav2Vec2 feature extractor object requires the following parameters to be instantiated:


---

feature_size: Speech models take a sequence of feature vectors as an input. While the length of this sequence obviously varies, the feature size should not. In the case of Wav2Vec2, the feature size is 1 because the model was trained on the raw speech signal

---

sampling_rate: The sampling rate at which the model is trained on.

---

padding_value: For batched inference, shorter inputs need to be padded with a specific value

---

do_normalize: Whether the input should be zero-mean-unit-variance normalized or not. Usually, speech models perform better when normalizing the input

---

return_attention_mask: Whether the model should make use of an attention_mask for batched inference. In general, models should always make use of the attention_mask to mask padded tokens. However, due to a very specific design choice of Wav2Vec2's "base" checkpoint, better results are achieved when using no attention_mask. This is not recommended for other speech models. For more information, one can take a look at this issue. Important If you want to use this notebook to fine-tune large-lv60, this parameter should be set to True.

In [ ]:
from transformers import Wav2Vec2FeatureExtractor

feature_extractor = Wav2Vec2FeatureExtractor(feature_size=1, sampling_rate=16000, padding_value=0.0, do_normalize=True, return_attention_mask=False)
### this is the feature extractor



In [ ]:
from transformers import Wav2Vec2Processor

processor = Wav2Vec2Processor(feature_extractor=feature_extractor, tokenizer=tokenizer)

In [ ]:
print(processor.feature_extractor)
processor.tokenizer.get_vocab()

Wav2Vec2FeatureExtractor {
  "do_normalize": true,
  "feature_extractor_type": "Wav2Vec2FeatureExtractor",
  "feature_size": 1,
  "padding_side": "right",
  "padding_value": 0.0,
  "return_attention_mask": false,
  "sampling_rate": 16000
}



{'S': 0,
 'U': 1,
 'X': 3,
 'P': 4,
 'K': 5,
 'V': 6,
 'B': 7,
 'Z': 8,
 'A': 9,
 'I': 10,
 'E': 11,
 'N': 12,
 'T': 13,
 'M': 14,
 'L': 15,
 'R': 16,
 'Y': 17,
 'G': 18,
 'W': 19,
 'F': 20,
 'C': 21,
 "'": 22,
 'D': 23,
 'O': 24,
 'J': 25,
 'Q': 26,
 'H': 27,
 '|': 2,
 '[UNK]': 28,
 '[PAD]': 29,
 '<s>': 30,
 '</s>': 31}

In [ ]:
#### lets check a few audio inputs

import IPython.display as ipd
import numpy as np
import random

rand_int = random.randint(0, len(train_split))

print(train_split[rand_int]["sentence"])
ipd.Audio(data=np.asarray(train_split[rand_int]["input_samples"]), autoplay=True, rate=16000)


THE ARTWORK WAS ALSO APPROVED AND DEVELOPED WITH THE BAND


# NOTE: Just ran the above code multiple times and teher are different accents in the data. Heard Indian, American, British, Autralian

In [ ]:
rand_int = random.randint(0, len(train_split))

print("Target text:", train_split[rand_int]["sentence"])
print("Input array shape:", np.asarray(train_split[rand_int]["input_samples"]).shape)


Target text: WHILE STUDYING IN YESHIVA HILDESHEIMER ALSO STUDIED CLASSICAL LANGUAGES
Input array shape: (87936,)


The output of `processor.feature_extractor` shows the configuration details of the **Wav2Vec2FeatureExtractor** used by the processor. Here's a breakdown of what each part means:

- **`do_normalize: true`**: This indicates that the audio signal will be normalized before feature extraction, i.e., the amplitude of the audio will be scaled to have a standard range.
  
- **`feature_extractor_type: "Wav2Vec2FeatureExtractor"`**: This specifies that the extractor being used is for the **Wav2Vec 2.0** model.

- **`feature_size: 1`**: This tells you the number of features per time step that the feature extractor will produce. In the case of Wav2Vec2, this is usually just 1 since the model is typically extracting raw waveform features.

- **`padding_side: "right"`**: When padding is required (for sequences of different lengths), this specifies that padding will be added to the right side of the input sequence.

- **`padding_value: 0.0`**: The value used for padding. Here, it is set to 0.

- **`return_attention_mask: false`**: This means that an attention mask will not be returned by the feature extractor, which is typically used to indicate which parts of the input are padded.

- **`sampling_rate: 16000`**: This indicates that the audio input is expected to have a **sampling rate of 16,000 Hz**, which is typical for speech-to-text models like Wav2Vec 2.0.



#Q] So which features are extracted?

The model takes the raw audio input, processes it through its layers, and outputs a **vector** (embedding) that represents the speech in a high-dimensional space. This embedding captures the relevant features of the speech, which can then be used for tasks like speech-to-text (ASR).

Note: The embeddings are the features vectors!

The error you're encountering (`TypeError: got multiple values for keyword argument 'return_attention_mask'`) suggests that the `processor` (likely a `Wav2Vec2Processor`) is being called with conflicting arguments. This happens because:

1. **The processor is trying to handle both audio and text processing** (since Wav2Vec2 processors combine a feature extractor and tokenizer).
2. **Somewhere internally, `return_attention_mask` is being passed twice** (either by default or explicitly).

### Solution: Explicitly Separate Audio and Text Processing
Since you're processing audio (`input_samples`) and text (`sentence`) separately, you should:

1. **Use `processor.feature_extractor` for audio** (no `return_attention_mask` needed).
2. **Use `processor.tokenizer` for text** (with `padding=False`, `truncation=False`).


1. **Separate `feature_extractor` and `tokenizer` calls**:
   - Audio: Uses `processor.feature_extractor` (requires `padding=True` for batching).
   - Text: Uses `processor.tokenizer` (no padding/truncation for CTC).

2. **Avoid implicit `return_attention_mask` conflicts**:
   - The error occurs because the combined `processor()` tries to handle both audio and text at once, leading to argument clashes. Explicit separation prevents this.

### Why This Works:
- CTC models **do not need padded labels** (the loss function handles alignment).
- Audio **must be padded/truncated** for batched processing (hence `padding=True` for the feature extractor).
- By splitting the processor into its components, we avoid internal argument conflicts.



NOTE: The below code takes xxx minutes

In [ ]:
def extract_features(batch):
    # Process audio (using feature_extractor directly)
    audio_features = processor.feature_extractor(
        batch["input_samples"],
        sampling_rate=16000
        #return_tensors="pt"
        # Note: The below anywas doesnt work since return_attention_mask = False
        #padding=True,  # Needed for audio (batched processing)
        #truncation=True,  # Needed for audio (long sequences)
        #max_length=160000
    )
    batch["input_samples"] = audio_features.input_values

    # Process text (using tokenizer directly, no padding/truncation for CTC)
    with processor.as_target_processor():
        labels_batch = processor.tokenizer(
            batch["sentence"]
            #return_tensors="pt",
            #padding=False,  # Disable padding for labels (CTC requirement)
            #truncation=False  # Disable truncation for labels
        )
    batch["labels"] = labels_batch.input_ids

    return batch


In [ ]:
train_split = train_split.map(extract_features, remove_columns=["client_id", "path", "sentence", "up_votes", "down_votes", "age", "gender", "accent", "locale", "segment", "input_length"])


Map:   0%|          | 0/75898 [00:00<?, ? examples/s]

/usr/local/lib/python3.11/dist-packages/transformers/models/wav2vec2/processing_wav2vec2.py:174: UserWarning: `as_target_processor` is deprecated and will be removed in v5 of Transformers. You can process your labels by using the argument `text` of the regular `__call__` method (either in the same call as your audio inputs, or in a separate call.
  warnings.warn(


In [ ]:
val_split = val_split.map(extract_features, remove_columns=["client_id", "path", "sentence", "up_votes", "down_votes", "age", "gender", "accent", "locale", "segment", "input_length"])


Map:   0%|          | 0/9487 [00:00<?, ? examples/s]

In [ ]:
test_split = test_split.map(extract_features, remove_columns=["client_id", "path", "sentence", "up_votes", "down_votes", "age", "gender", "accent", "locale", "segment", "input_length"])

Map:   0%|          | 0/9488 [00:00<?, ? examples/s]

In [ ]:
import os

# Create a directory inside /content
dataset_dir = '/content/processed_data/train'
os.makedirs(dataset_dir, exist_ok=True)

# Save the dataset to the directory
train_split.save_to_disk(dataset_dir)

print(f"Dataset saved to: {dataset_dir}")

Saving the dataset (0/56 shards):   0%|          | 0/75898 [00:00<?, ? examples/s]

Dataset saved to: /content/processed_data/train


In [ ]:
# Create a directory inside /content
dataset_dir = '/content/processed_data/val'
os.makedirs(dataset_dir, exist_ok=True)

# Save the dataset to the directory
val_split.save_to_disk(dataset_dir)

print(f"Dataset saved to: {dataset_dir}")

Saving the dataset (0/7 shards):   0%|          | 0/9487 [00:00<?, ? examples/s]

Dataset saved to: /content/processed_data/val


In [ ]:
# Create a directory inside /content
dataset_dir = '/content/processed_data/test'
os.makedirs(dataset_dir, exist_ok=True)

# Save the dataset to the directory
test_split.save_to_disk(dataset_dir)

print(f"Dataset saved to: {dataset_dir}")

Saving the dataset (0/7 shards):   0%|          | 0/9488 [00:00<?, ? examples/s]

Dataset saved to: /content/processed_data/test


In [ ]:
#### download to local
import shutil
from google.colab import files

# Path to the saved dataset directory
dataset_dir = '/content/processed_data'

# Create a zip file of the dataset directory
shutil.make_archive('/content/processed_data', 'zip', dataset_dir)

'/content/processed_data.zip'

In [ ]:
import shutil

source_path = './processed_data.zip'
destination_path = '/content/drive/My Drive/Speech_to_text/dataset_10/processed_data.zip'

shutil.move(source_path, destination_path)

print(f"File successfully moved to: {destination_path}")


File successfully moved to: /content/drive/My Drive/Speech_to_text/dataset_10/processed_data.zip


In [ ]:
### check the data

# Unzip the file
import zipfile

# Specify the path to your file in Drive
zip_file_path = '/content/drive/My Drive/Speech_to_text/dataset_10/processed_data.zip'


with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
    zip_ref.extractall('/content/dataset/')

In [ ]:
from datasets import load_from_disk


# Load the dataset from the saved directory
train = load_from_disk('/content/dataset/train')

val = load_from_disk('/content/dataset/val')

test = load_from_disk('/content/dataset/test')

Loading dataset from disk:   0%|          | 0/56 [00:00<?, ?it/s]

In [ ]:
print(train)
print(val)
print(test)


Dataset({
    features: ['input_samples', 'labels'],
    num_rows: 75898
})
Dataset({
    features: ['input_samples', 'labels'],
    num_rows: 9487
})
Dataset({
    features: ['input_samples', 'labels'],
    num_rows: 9488
})
